In [1]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,SimpleRNN,LSTM

from sklearn.model_selection import train_test_split

In [3]:
df=pd.read_csv("C:/Users/User/Desktop/HAM_or_SPAM_TF_Version/SPAM text message 20170820 - Data.csv")
df.head(10)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
def classes(category):
        if(category=='ham'):
            return 1
        else:
            return 0
        
df['Category']=df['Category'].apply(classes)
df.tail(10)

,Category,Message
5562,1,Ok lor... Sony ericsson salesman... I ask shuh...
5563,1,Ard 6 like dat lor.
5564,1,Why don't you wait 'til at least wednesday to ...
5565,1,Huh y lei...
5566,0,REMINDER FROM O2: To get 2.50 pounds free call...
5567,0,This is the 2nd time we have tried 2 contact u...
5568,1,Will ü b going to esplanade fr home?
5569,1,"Pity, * was in mood for that. So...any other s..."
5570,1,The guy did some bitching but I acted like i'd...
5571,1,Rofl. Its true to its name


In [5]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [6]:
df.describe()

,Category
count,5572.000000
mean,0.865937
std,0.340751
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
messages=np.asarray(df['Message'])

In [8]:
messages

array(['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'Ok lar... Joking wif u oni...',
       "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ..., 'Pity, * was in mood for that. So...any other suggestions?',
       "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'Rofl. Its true to its name'], dtype=object)

In [9]:
classes=np.asarray(df['Category'])
classes

array([1, 1, 0, ..., 1, 1, 1], dtype=int64)

In [10]:
final=[]

def total(messages):
    for i in messages:
        final.append(messages)

df['Message'].nunique()

5157

In [11]:

tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(messages)
sequences = tokenizer.texts_to_sequences(messages)

max=0

sequences[0]

[49,
 471,
 4426,
 840,
 755,
 658,
 65,
 8,
 1327,
 88,
 123,
 351,
 1328,
 148,
 2993,
 1329,
 67,
 58,
 4427,
 144]

In [12]:
for i in sequences:
    h=len(i)
    if(h>max):
        max=h
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=189)

X=data
data

array([[   0,    0,    0, ...,   58, 4427,  144],
       [   0,    0,    0, ...,  472,    6, 1940],
       [   0,    0,    0, ...,  660,  391, 2994],
       ...,
       [   0,    0,    0, ...,  107,  250, 9003],
       [   0,    0,    0, ...,  198,   12,   47],
       [   0,    0,    0, ...,    2,   61,  267]])

In [13]:

X.shape

(5572, 189)

In [14]:

Y=df['Category']
Y=keras.utils.to_categorical(Y)
Y

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(5014, 189) (5014, 2)
(558, 189) (558, 2)


In [16]:
model = Sequential()
model.add(Embedding(input_dim=10000,output_dim=32,input_length=189))
model.add(SimpleRNN(units=32))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
 metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 189, 32)           320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 2)                 66        
Total params: 322,146
Trainable params: 322,146
Non-trainable params: 0
_________________________________________________________________


In [18]:
batch_size = 60
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, validation_split=0.2)

Epoch 1/5
67/67 [==============================] - 2s 27ms/step - loss: 0.1595 - acc: 0.9678 - val_loss: 0.1759 - val_acc: 0.9362
Epoch 2/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0677 - acc: 0.9825 - val_loss: 0.0884 - val_acc: 0.9761
Epoch 3/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0359 - acc: 0.9910 - val_loss: 0.0785 - val_acc: 0.9761
Epoch 4/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0217 - acc: 0.9940 - val_loss: 0.0809 - val_acc: 0.9781
Epoch 5/5
67/67 [==============================] - 2s 26ms/step - loss: 0.0249 - acc: 0.9908 - val_loss: 0.1168 - val_acc: 0.9701


In [19]:
acc = model.evaluate(X_test,Y_test)
print("LOSS => {0:.2f} || AUC => {1:.2f} ".format(acc[0],acc[1]))

18/18 [==============================] - 0s 5ms/step - loss: 0.0831 - acc: 0.9785
LOSS => 0.08 || AUC => 0.98 


In [20]:
def message_to_array(msg):
    msg = msg.lower().split(' ')
    test_seq = np.array([word_index[word] for word in msg])
    test_seq = np.pad(test_seq, (189-len(test_seq), 0),
    'constant', constant_values=(0))
    test_seq = test_seq.reshape(1, 189)
    return test_seq

In [22]:

custom_msg = 'Congratulations ur awarded 500 of CD vouchers or 125gift guaranteed and completely free entry for movies'
test_seq = message_to_array(custom_msg)
pred = model.predict_classes(test_seq)

if pred==0:
    print("Spam!")
else:
    print("Ham!")

Ham!


In [23]:
custom_msg = 'Okay fine i will be coming with you we will be having a great time together'
test_seq = message_to_array(custom_msg)
pred = model.predict_classes(test_seq)

if pred==0:
    print("Spam!")
else:
    print("Ham!")

Ham!


In [24]:
model1 = Sequential()
model1.add(Embedding(input_dim=10000,output_dim=32,input_length=189))
model1.add(LSTM(200))
model1.add(Dense(2,activation='sigmoid'))
model1.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [25]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 189, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 200)               186400    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 506,802
Trainable params: 506,802
Non-trainable params: 0
_________________________________________________________________


In [26]:
batch_size = 60
model1.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, validation_split=0.2)

Epoch 1/3
67/67 [==============================] - 16s 233ms/step - loss: 0.3553 - accuracy: 0.8783 - val_loss: 0.1654 - val_accuracy: 0.9611
Epoch 2/3
67/67 [==============================] - 19s 277ms/step - loss: 0.0653 - accuracy: 0.9843 - val_loss: 0.0671 - val_accuracy: 0.9850
Epoch 3/3
67/67 [==============================] - 19s 285ms/step - loss: 0.0313 - accuracy: 0.9910 - val_loss: 0.0659 - val_accuracy: 0.9840


In [27]:
custom_msg = 'YO FUCK YOU'
test_seq = message_to_array(custom_msg)
pred = model.predict_classes(test_seq)

if pred==0:
    print("Spam!")
else:
    print("Ham!")

Ham!


In [28]:
custom_msg1 = 'I got another job The one at the hospital doing data analysis or something starts on Monday Not sure when my thesis will finish'
test_seq = message_to_array(custom_msg1)
pred = model1.predict_classes(test_seq)

if pred==0:
    print("Spam!")
else:
    print("Ham!")

Ham!


<b>CONVERT INTO TFLITE MODEL!</b>
--

In [39]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpgg6ms14v\assets


INFO:tensorflow:Assets written to: C:\Users\User\AppData\Local\Temp\tmpgg6ms14v\assets


In [40]:
with tf.io.gfile.GFile('model.tflite', 'wb') as f:
    open("C:/Users/User/Desktop/HAM_or_SPAM_TF_Version/model.tflite", "wb").write(tflite_model)